# Converting the PDF to Text

The PDFs are converted to text using this package: https://github.com/jsvine/pdfplumber#extracting-text. Follow the installation instructions before running this notebook. 


In [13]:
import re 
import pdfplumber

full_pdf_text = ""

# input the path to the PDF you want to convert here
with pdfplumber.open("foucault.pdf") as pdf:
    for i in range(len(pdf.pages)):
        full_page = pdf.pages[i].extract_text()

        first_line = full_page.split("\n")[0] # where page numbers at the top of the page are likely to be found
        last_line = full_page.split("\n")[-1] # where page numbers at the bottom of the page are likely to be found
        full_pdf_text += full_page

        if len(full_page) != 0:
            top_left_page_matches = re.findall("^([xXvViIlL]+|\d{1,3})\s", first_line)
            top_right_page_matches = re.findall("\s([xXvViIlL]+|\d{1,3})$", first_line)
            bottom_page_matches = re.findall("^([xXvViIlL]+|\d{1,3})$", last_line)
            
            all_matches = []
            if top_left_page_matches:
                all_matches.append(top_left_page_matches[0].strip())
            if top_right_page_matches:
                all_matches.append(top_right_page_matches[-1].strip())
            if bottom_page_matches:
                all_matches.append(bottom_page_matches[-1].strip())
            if len(all_matches) > 0:
                if len(all_matches) > 1: # more than one potential page number 
                    print(f'\n{len(all_matches)} potential page numbers were found for this page.')
                    print(all_matches)
                    print(f'Check PDF page {i+1} for the correct page number and enter it below (exactly as displayed in the print) If the page has no printed page number, do not input anything.')
                    correct_page_number = input()
                    print(correct_page_number)
                    if len(correct_page_number.strip()) != 0:
                        printed_page_number = correct_page_number.strip()
                    else:
                        printed_page_number = None
                else:
                    printed_page_number = all_matches[0]

                if printed_page_number is not None: 
                    full_pdf_text += f'\n\n~printed_page_number:{printed_page_number}~'
                
        full_pdf_text += f'\n~indexed_page_number:{i+1}~\n\n'

# enter the name of your output file here     
with open("kuhn_output.txt", "w") as text_file:
    text_file.write(full_pdf_text)


[]
[]
[]
[]
['vi']
['I']
[]
['4']
['5']
['6']
['7']
['8']
['9']
['10']
['11']
['12']
['13']
['14']
['15']
['16']
['17']
['18']
['19']
['II']
[]
['24']
['25']
['26']
['27']
['28']
['29']
['30']
['31']
['32']
['33']
[]
['35']
['36']
['37']
['38']
['39']
['40']
['41']
['42']
['43']
[]
['45']
['46']
['47']
['48']
['49']
['50']
['51']
['52']
['53']
['54']
[]
['56']
['57']
['58']
['59']
['60']
['61']
[]
['63']
['64']
['65']
['66']
['67']
['68']
['69']
['70']
[]
['72']
['73']
['74']
['75']
['76']
['77']
['78']
[]
['80']
['81']
['82']
['83']
['84']
['85']
['III']
[]
['90']
['91']
['92']
['93']
['94']
['95']
['96']
['97']
['98']
[]
['100']
['101']
['102']
['103']
['104']
['105']
['106']
['107']
['108']
['109']
['110']
['111']
['112']
['113']
['114']
['115']
['116']
['117']
['118']
[]
['120']
['121']
['122']
['123']
['124']
['125']
['126']
['127']
['128']
['129']
['130']
['131']
['132']
[]
['134']
['135']
['136']
['137']
['138']
['139']
['140']
['141']
[]
['143']
['144']
['145']
['146']
['147']
